In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install ultralytics scikit-learn matplotlib

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 43.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 119.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 50.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 103.2 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstallin

In [ ]:
# 0. 필수 라이브러리 설치
!pip install ultralytics scikit-learn matplotlib

# 1. 라이브러리 임포트
from ultralytics import YOLO
import matplotlib.pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
import os
from pathlib import Path
import matplotlib.pyplot as plt

# 2. 데이터셋 구조 설정
"""
데이터셋 폴더 구조 (반드시 준수):
dataset/
├── train/
│   ├── flooded/     # 침수 이미지 (예: 0001.jpg, 0002.jpg...)
│   └── non_flooded/ # 정상 이미지
└── val/
    ├── flooded/
    └── non_flooded/
"""

# 3. 모델 초기화 (분류 작업용 사전훈련 모델)
model = YOLO('yolov8n-cls.pt')  # yolov8s-cls, yolov8m-cls 등으로 변경 가능

# 4. 하이퍼파라미터 설정
training_params = {
    'data': '/content/drive/MyDrive/FloodDetect.v1i.yolov8',
    'epochs': 50,
    'batch': 64,  # GPU 메모리 따라 조정 (32-128)
    'imgsz': 640,  # 높은 해상도 권장 (224, 384, 640)
    'lr0': 0.001,
    'optimizer': 'AdamW',
    'device': '0',  # GPU 사용 시 '0', CPU 시 'cpu'
    'augment': True,  # 자동 데이터 증강 활성화
    'seed': 42
}

# 5. 모델 훈련
results = model.train(**training_params)

# 6. 성능 평가
# 혼동 행렬 시각화
val_results = model.val()
y_true = val_results.labels.tolist()
y_pred = val_results.pred.tolist()

plt.figure(figsize=(8,6))
sns.heatmap(confusion_matrix(y_true, y_pred),
            annot=True, fmt='d',
            cmap='Blues',
            xticklabels=['non_flooded', 'flooded'],
            yticklabels=['non_flooded', 'flooded'])
plt.xlabel('Predicted')
plt.ylabel('Actual')
plt.title('Confusion Matrix')
plt.show()

# 분류 보고서 출력
print(classification_report(y_true, y_pred, target_names=['non_flooded', 'flooded']))

# 7. 모델 저장
model.export(format='onnx')  # ONNX 형식으로 추출 (옵션)
model.save('best_flood_classifier.pt')

# 9. 테스트 데이터셋 성능 평가
def evaluate_test_set(test_dir):
    class_names = ['non_flooded', 'flooded']

    # 각 클래스별 테스트 폴더 평가
    all_preds = []
    all_labels = []

    for class_idx, class_name in enumerate(class_names):
        test_folder = f"{test_dir}/{class_name}"
        if not os.path.exists(test_folder):
            print(f"경고: {test_folder} 경로가 존재하지 않습니다.")
            continue

        # 해당 클래스 폴더의 모든 이미지 예측
        results = model.predict(test_folder)

        # 예측 결과와 실제 라벨 수집
        for result in results:
            pred_class = result.probs.top1
            all_preds.append(pred_class)
            all_labels.append(class_idx)  # 폴더명이 실제 클래스

    # 혼동 행렬 시각화
    plt.figure(figsize=(8,6))
    cm = confusion_matrix(all_labels, all_preds)
    sns.heatmap(cm,
                annot=True, fmt='d',
                cmap='Blues',
                xticklabels=class_names,
                yticklabels=class_names)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Test Set Confusion Matrix')
    plt.show()

    # 분류 보고서 출력
    print("\n테스트 데이터셋 성능 평가:")
    print(classification_report(all_labels, all_preds, target_names=class_names))

    # 테스트 및 검증 데이터셋 성능 지표 계산
    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

    test_metrics = {
        'Accuracy': accuracy_score(all_labels, all_preds),
        'Precision': precision_score(all_labels, all_preds, average='binary', pos_label=1),
        'Recall': recall_score(all_labels, all_preds, average='binary', pos_label=1),
        'F1-Score': f1_score(all_labels, all_preds, average='binary', pos_label=1)
    }

    val_metrics = {
        'Accuracy': accuracy_score(y_true, y_pred),
        'Precision': precision_score(y_true, y_pred, average='binary', pos_label=1),
        'Recall': recall_score(y_true, y_pred, average='binary', pos_label=1),
        'F1-Score': f1_score(y_true, y_pred, average='binary', pos_label=1)
    }

    # 성능 지표 비교 시각화
    plt.figure(figsize=(10, 6))
    import numpy as np
    x = np.arange(len(test_metrics))
    width = 0.35

    plt.bar(x - width/2, list(test_metrics.values()), width, label='Test')
    plt.bar(x + width/2, list(val_metrics.values()), width, label='Validation')

    plt.xticks(x, list(test_metrics.keys()))
    plt.ylim(0, 1.1)
    plt.ylabel('Score')
    plt.title('Performance Metrics Comparison')
    plt.legend()
    plt.show()

    return all_labels, all_preds

# 10. 테스트 데이터셋 평가 실행
print("\n===== 테스트 데이터셋 성능 평가 =====")
test_labels, test_preds = evaluate_test_set('/content/drive/MyDrive/FloodDetect.v1i.yolov8/test')

# 11. 모델 학습 곡선 시각화
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)
plt.plot(results.results_dict['train/loss'], label='Train Loss')
plt.plot(results.results_dict['val/loss'], label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Loss Curve')
plt.legend()

plt.subplot(1, 2, 2)
plt.plot(results.results_dict['metrics/accuracy_top1'], label='Top-1 Accuracy')
plt.plot(results.results_dict['val/accuracy_top1'], label='Val Top-1 Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy Curve')
plt.legend()
plt.tight_layout()
plt.show()

# 12. ROC 곡선 및 AUC 계산
from sklearn.metrics import roc_curve, auc

# Validation 및 Test 데이터셋에 대한 ROC 곡선
def plot_roc_curves():
    # Validation 데이터셋
    val_flooded_results = model.predict('/content/drive/MyDrive/FloodDetect.v1i.yolov8/val/flooded')
    val_non_flooded_results = model.predict('/content/drive/MyDrive/FloodDetect.v1i.yolov8/val/non_flooded')

    val_probs = [result.probs.data[1].item() for result in val_flooded_results] + \
               [result.probs.data[1].item() for result in val_non_flooded_results]

    val_true = [1] * len(val_flooded_results) + \
              [0] * len(val_non_flooded_results)

    fpr_val, tpr_val, _ = roc_curve(val_true, val_probs)
    roc_auc_val = auc(fpr_val, tpr_val)

    # Test 데이터셋
    test_flooded_results = model.predict('/content/drive/MyDrive/FloodDetect.v1i.yolov8/test/flooded')
    test_non_flooded_results = model.predict('/content/drive/MyDrive/FloodDetect.v1i.yolov8/test/non_flooded')

    test_probs = [result.probs.data[1].item() for result in test_flooded_results] + \
               [result.probs.data[1].item() for result in test_non_flooded_results]

    test_true = [1] * len(test_flooded_results) + \
              [0] * len(test_non_flooded_results)

    fpr_test, tpr_test, _ = roc_curve(test_true, test_probs)
    roc_auc_test = auc(fpr_test, tpr_test)

    # ROC 곡선 시각화
    plt.figure(figsize=(8, 6))
    plt.plot(fpr_val, tpr_val, lw=2, label=f'Validation ROC (AUC = {roc_auc_val:.3f})')
    plt.plot(fpr_test, tpr_test, lw=2, label=f'Test ROC (AUC = {roc_auc_test:.3f})')
    plt.plot([0, 1], [0, 1], 'k--', lw=2)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc="lower right")
    plt.show()

# ROC 곡선 그리기
plot_roc_curves()


Ultralytics 8.3.133 🚀 Python-3.11.12 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=True, auto_augment=randaugment, batch=64, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/drive/MyDrive/FloodDetect.v1i.yolov8, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n-cls.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train2, nbs=64, nms=False, opset=None, optimize=False, optimizer=AdamW, overlap_mask=True, patience=100, perspective=0.0, plots=True, p

100%|██████████| 5.35M/5.35M [00:00<00:00, 132MB/s]


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.5±0.1 ms, read: 0.1±0.0 MB/s, size: 67.4 KB)


train: Scanning /content/drive/MyDrive/FloodDetect.v1i.yolov8/train... 2292 images, 0 corrupt: 100%|██████████| 2292/2292 [00:55<00:00, 41.13it/s] 


train: New cache created: /content/drive/MyDrive/FloodDetect.v1i.yolov8/train.cache
val: Fast image access ✅ (ping: 1.3±1.7 ms, read: 0.1±0.0 MB/s, size: 72.1 KB)


...

val: Scanning /content/drive/MyDrive/FloodDetect.v1i.yolov8/test... 135 images, 0 corrupt: 100%|██████████| 135/135 [00:00<?, ?it/s]
               classes   top1_acc   top5_acc:   0%|          | 0/3 [00:00<?, ?it/s]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc:  33%|███▎      | 1/3 [00:06<00:13,  6.99s/it]

WARNING ⚠️ ClassificationModel does not support 'augment=True' prediction. Reverting to single-scale prediction.


               classes   top1_acc   top5_acc:  67%|██████▋   | 2/3 [00:10<00:04,  4.87s/it]